# Graphing Inputs/Outputs of Pre and Post-Synaptic Neurons

## Import all relevent libraries

In [ ]:
#Import all relevent libraries
from caveclient import CAVEclient
import nglui
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from phase3 import nda

## Using CAVEclient to query data

In [ ]:
client = CAVEclient()

In [ ]:
client.auth.save_token(token="a80f7690d93ead1b46e980628df5beae",overwrite = True)

In [ ]:
# this is the datastack name of the public release
# passing it will auto-configure many of the services
client = CAVEclient('minnie65_public_v343')

In [ ]:
client.materialize.get_tables()

## Filter data to obtain depth

In [ ]:
df_fun_coreg=client.materialize.query_table('functional_coreg') #dataframe of the neurons that have functional data available

In [ ]:
df_fun_coreg.head()

In [ ]:
seg_id = 864691136136642173 #define a single neuron to look at pre/post synaptic connections
#we will iterate over pre-synaptic cells that are present in functionally matched data set, 
#we identify them based on their unique pt_root_id and for each pre-synaptic cell identified this way,
#we consider the synapses (from synapse dataset) to which these cells are pre-synaptic
#for each id of neuron, we therefore first consider all synapses to which it is presynaptic and obtaining these
#synapses we look at neurons post-synaptic to them
#we create a dataframe
#we plot using this
#input and output depth - stackunit_id of pre-/post-synaptic cell
#TO GENERALIZE THIS PROCESS
seg_ids = df_fun_coreg.iloc[:, 3] #extract a column
seg_ids

In [ ]:
# lets get the outputs of this cell
#output_df = client.materialize.synapse_query(pre_ids=seg_id)
# and the inputs

input_df = client.materialize.synapse_query(post_ids=seg_id)
seg_ids = df_fun_coreg.iloc[:, 3] #extract a column

#seg_ids #extract only ids which are in the functional dataset, we will consider them as postsynaptic id's
input_depth=np.array([])
output_depth=np.array([])
for id in seg_ids: # iterate over postsynaptic id's present in functional data
    input_df=client.materialize.synapse_query(post_ids=seg_id) #synapses and their relevant data whose post-synaptic id appears in functional data
    input_df.rename(columns = {'pre_pt_root_id':'pt_root_id'}, inplace = True) #rename to be able to match based on primary key of functional dataset in the next step
    #it will be used to match with foreign key of input_df - presynaptic id
    fun_coreg_matched_df = pd.merge(input_df[['pt_root_id','post_pt_root_id']],df_fun_coreg[['pt_root_id','session', 'scan_idx', 'unit_id']],on='pt_root_id',how='inner')
# merging the relevant columns from input_df and df_fun_coreg dataframes based on pt_root_id (only pt_root_id present in fun_coreg dataframe is considered)
    #fun_coreg_matched_df contains data on synapses with postsynaptic cell's id equal to seg_id (corresponding to this iteration of the loop) and
    #presynaptic id's only of cells present in functional dataset
    scan_keys = []
    dict1 = {}
    for i in range(len(fun_coreg_matched_df)):
        dict1 = {'session': fun_coreg_matched_df.loc[i].at['session'], 'scan_idx': fun_coreg_matched_df.loc[i].at['scan_idx'], 'unit_id': fun_coreg_matched_df.loc[i].at['unit_id']}
        scan_keys.append(dict1)
    filtered_stackunit=nda.StackUnit() & scan_keys
    stack_z_values = np.stack((filtered_stackunit).fetch("stack_z"), -1) #extracting relevant information
    stack_z_values=np.array(stack_z_values)
    n_presynaptic=len(stack_z_values)
    wrong_y_coordinates=np.repeat(np.average(stack_z_values), n_presynaptic) #they are arbitrary but we have the same
    #postsynaptic cell for a group of presynaptic cells considered in this iteration so we have array of repeated elements
    #the same position as we are dealing with the same post-synaptic cell (but idk how to get its coordinate so I will
    #take some arbitrary number - it will still be good for visualization as we take arbitrary for all postsynaptic positiosn
    #- not fully arbitrary though as we take average of presynaptic positions assuming that postsynaptic cells are relatively close 
    input_depth=np.concatenate((input_depth, stack_z_values))
    output_depth=np.concatenate((output_depth, wrong_y_coordinates))
    
output_depth
input_depth
plt.plot(input_depth, output_depth)

    

In [ ]:
# here are the number of output synapses and input synapses
input_df
len(output_df), len(input_df)

In [ ]:
# take peak at the input synapse dataframe
input_df.head()

In [ ]:
input_df.rename(columns = {'pre_pt_root_id':'pt_root_id'}, inplace = True) #
input_df.head()

In [ ]:
fun_coreg_matched_df = pd.merge(input_df[['pt_root_id','post_pt_root_id']],df_fun_coreg[['pt_root_id','session', 'scan_idx', 'unit_id']],on='pt_root_id',how='inner')
# merging the relevant columns from input_df and df_fun_coreg dataframes based on pt_root_id (only pt_root_id present in fun_coreg dataframe is considered)
fun_coreg_matched_df.head()
len(fun_coreg_matched_df)

In [ ]:
fun_coreg_matched_df.loc[0].at['session']

In [ ]:
print(len(input_df),len(fun_coreg_matched_df)) # input_df contains synapses for which post-synaptic id 

In [ ]:
scan_keys = []
dict1 = {}
for i in range(len(fun_coreg_matched_df)):
    dict1 = {'session': fun_coreg_matched_df.loc[i].at['session'], 'scan_idx': fun_coreg_matched_df.loc[i].at['scan_idx'], 'unit_id': fun_coreg_matched_df.loc[i].at['unit_id']}
    scan_keys.append(dict1)
    
print(scan_keys)
    

In [ ]:
filtered_stackunit=nda.StackUnit() & scan_keys

In [ ]:
filtered_stackunit 

In [ ]:
stack_z_values = np.stack((filtered_stackunit).fetch("stack_z"), -1)

In [ ]:
stack_z_values

In [ ]:
nda.StackUnit()